<a href="https://colab.research.google.com/github/abhishekmishra2612/Pneumonia-Detection/blob/master/PNEUMONIA_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhishekmishra2612","key":"8b2e61bd8a7055cb301212c8d5ea66e0"}'}

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:29<00:00, 38.9MB/s]
100% 2.29G/2.29G [00:29<00:00, 82.2MB/s]


In [0]:
from zipfile import ZipFile
zip_file = ZipFile('/content/chest-xray-pneumonia.zip')

In [0]:
with ZipFile('/content/chest-xray-pneumonia.zip',"r") as zip:
  zip.extractall() 

In [0]:
a=[]
while True:
  a.append(1)

In [0]:
from keras.preprocessing.image import load_img,img_to_array
from keras.applications.inception_v3 import InceptionV3,preprocess_input

feature_extracter=InceptionV3()
feature_extracter.layers.pop()

In [0]:
feature_extracter.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
_______________________________________________________________________________________

In [0]:
from keras.models import Model
for layer in feature_extracter.layers:
  layer.trainable=False

feature_extracter=Model(inputs=feature_extracter.input,outputs=feature_extracter.layers[-1].output)

In [0]:
import os
dirname_1="/content/chest_xray/train/NORMAL/"
train_NORMAL=[]
for f in os.listdir("/content/chest_xray/train/NORMAL"):
  image=preprocess_input(img_to_array(load_img(dirname_1+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  train_NORMAL.append(pred)
  
dirname_2="/content/chest_xray/train/PNEUMONIA/" 
train_PNEUMONIA=[]
for f in os.listdir("/content/chest_xray/train/PNEUMONIA"):
  image=preprocess_input(img_to_array(load_img(dirname_2+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  train_PNEUMONIA.append(pred)


In [0]:
import numpy as np
x_train=np.array(train_NORMAL+train_PNEUMONIA)

In [0]:
x_train=x_train.reshape(5216,2048)

In [0]:
x_train.shape

(5216, 2048)

In [0]:
np.save("/content/drive/My Drive/PNEUMONIA_features.npy",x_train)

In [0]:
y_train=[]
for _ in range(len(train_NORMAL)):
  y_train.append(0)
for _ in range(len(train_PNEUMONIA)):
  y_train.append(1)  

In [0]:
y_train=np.array(y_train)
y_train.shape

(5216,)

In [0]:
np.save("/content/drive/My Drive/PNEUMONIA_labels.npy",y_train)

In [0]:
import os
dirname_1="/content/chest_xray/val/NORMAL/"
val_NORMAL=[]
for f in os.listdir("/content/chest_xray/val/NORMAL"):
  image=preprocess_input(img_to_array(load_img(dirname_1+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  val_NORMAL.append(pred)
  
dirname_2="/content/chest_xray/val/PNEUMONIA/" 
val_PNEUMONIA=[]
for f in os.listdir("/content/chest_xray/val/PNEUMONIA"):
  image=preprocess_input(img_to_array(load_img(dirname_2+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  val_PNEUMONIA.append(pred)

In [0]:
x_val=np.array(val_NORMAL+val_PNEUMONIA)

In [0]:
x_val=x_val.reshape(16,2048)

In [0]:
np.save("/content/drive/My Drive/PNEUMONIA_features_val.npy",x_val)

In [0]:
y_val=[]
for _ in range(len(val_NORMAL)):
  y_val.append(0)
for _ in range(len(val_PNEUMONIA)):
  y_val.append(1)
y_val=np.array(y_val) 
np.save("/content/drive/My Drive/PNEUMONIA_labels_val.npy",y_val) 

In [0]:
dirname_1="/content/chest_xray/test/NORMAL/"
test_NORMAL=[]
for f in os.listdir("/content/chest_xray/test/NORMAL"):
  image=preprocess_input(img_to_array(load_img(dirname_1+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  test_NORMAL.append(pred)
  
dirname_2="/content/chest_xray/test/PNEUMONIA/" 
test_PNEUMONIA=[]
for f in os.listdir("/content/chest_xray/test/PNEUMONIA"):
  image=preprocess_input(img_to_array(load_img(dirname_2+f,target_size=(299,299))))
  pred=feature_extracter.predict(image.reshape(1,299,299,3))
  test_PNEUMONIA.append(pred)

In [0]:
x_test=np.array(test_NORMAL+test_PNEUMONIA)

In [0]:
x_test=x_test.reshape(624,2048)

In [0]:
np.save("/content/drive/My Drive/PNEUMONIA_features_test.npy",x_test)

In [0]:
y_test=[]
for _ in range(len(test_NORMAL)):
  y_test.append(0)
for _ in range(len(test_PNEUMONIA)):
  y_test.append(1)
y_test=np.array(y_test) 
np.save("/content/drive/My Drive/PNEUMONIA_labels_test.npy",y_test)